# This notebook is downloaded from Google colab. DO NOT RUN IT! The results will be gone otherwise.

In [0]:
import glob
import pickle
import numpy as np
import os
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, CuDNNLSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Step 0: Use google colab

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
notes = np.load('/content/gdrive/My Drive/notes.npy') #Get file preprocessed in MIDI Preprocessing.ipynb

## Step 1: Preprocess data

In [0]:
pitchnames = sorted(list(set(notes)))
data_size, vocab_size = len(notes), len(pitchnames)
print('data has %d notes, %d unique pitches.' % (data_size, vocab_size))

data has 254731 notes, 286 unique pitches.


In [0]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

### All the way up to here, every step is the same as what I did in the ABC.ipynb notebook

In [ ]:
seq_len = 100
dataX=[]
dataY=[]
for i in range(0, len(notes) - seq_len, 1):
    x = notes[i:i + seq_len]
    y = notes[i + seq_len]        
    dataX.append([note_to_int[char] for char in x])
    dataY.append(note_to_int[y])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

This time, I'm doing a many-to-one LSTM: using 100 notes to predict the next one.

In [0]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_len, 1))
# normalize
X = X / float(vocab_size)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Step 2: build the model

In [0]:
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(X.shape[1], X.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(CuDNNLSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          2101248   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

## Step 3: train the model and generate MIDI format music along the way

In [0]:
from keras.callbacks import *
filepath="/content/gdrive/My Drive/acc:{acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

This is the callbacks part I talked about in my blog. It's very important to train and save the model simaltaneously especially when the model is trained intermittently.

In [0]:
def generate_notes(model, epoch):
    start = np.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    print ("Seed:")
    print ("\"", ''.join([int_to_note[value] for value in pattern]), "\"")
    output = []
# generate 1000 characters 1 by 1
    for i in range(1000):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_size)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        output.append(result)
        
        pattern.append(index) # the previous output is now appended to the input to predict the next one
        pattern = pattern[1:len(pattern)]
        
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes) #create score
    midi_file_name = ('/content/gdrive/My Drive/lstm_out_{}.mid'.format(epoch)) #write the score to midi
    midi_stream.write('midi', fp=midi_file_name)

   

    return ("\nDone.")

In [0]:
Epoch = 0
while Epoch <= 30:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch) #generate a piece of music every 10 epochs
    Epoch+=1




0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1





254631/254631 [==============================] - 400s 2ms/step - loss: 4.6353 - acc: 0.0233

Epoch 00001: acc improved from -inf to 0.02331, saving model to /content/gdrive/My Drive/acc:0.023.hdf5
Seed:
" C3D3E3F3C55.8.0G3G#3C510.0.4G3F3C510.0.4E3C#3C510.0.4C3B-2C510.0.4G#2G2C59.0.36A3C4E-4F#4A4C5E-5F#5A5C6E-6F#6E-6C6A5F#5E-5C5A4F#4E-4C4A3F#3E-3C3A2F#2E-2C2A1G#1A1C2E-2G#1A1C2E-2G#1A1C2E-28.9A1C2E-2F1A3C4E-4F#4G#4F#4F4E-4F4E-4C#4C4C#4E-4C4F2A3F1C4E-4F#4G#4F#4F4E-4 "



1
Epoch 1/1
254631/254631 [==============================] - 390s 2ms/step - loss: 4.4390 - acc: 0.0268

Epoch 00001: acc improved from 0.02331 to 0.02676, saving model to /content/gdrive/My Drive/acc:0.027.hdf5



2
Epoch 1/1
254631/254631 [==============================] - 390s 2ms/step - loss: 4.4038 - acc: 0.0279

Epoch 00001: acc improved from 0.02676 to 0.02787, saving model to /content/gdrive/My Driv

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.235.hdf5') #reload the weights when the runtime stops

In [0]:
Epoch = 24 #similarly....
while Epoch <= 30:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




24
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
254631/254631 [==============================] - 397s 2ms/step - loss: 2.9945 - acc: 0.2396

Epoch 00001: acc improved from -inf to 0.23955, saving model to /content/gdrive/My Drive/acc:0.240.hdf5



25
Epoch 1/1
254631/254631 [==============================] - 400s 2ms/step - loss: 2.9649 - acc: 0.2467

Epoch 00001: acc improved from 0.23955 to 0.24674, saving model to /content/gdrive/My Drive/acc:0.247.hdf5



26
Epoch 1/1
254631/254631 [==============================] - 403s 2ms/step - loss: 2.9442 - acc: 0.2484

Epoch 00001: acc improved from 0.24674 to 0.24841, saving model to /content/gdrive/My Drive/acc:0.248.hdf5



27
Epoch 1/1
254631/254631 [==============================] - 405s 2ms/step - loss: 2.9202 - acc: 0.2557

Epoch 00001: acc improved from 0.24841 to 0.25569, saving model to /content/gdrive/My Drive/acc:0.256.hdf5



28
Epoch 1/1
254631/254631 [============

### And so on and on...

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.268.hdf5')

In [0]:
Epoch = 31
while Epoch <= 100:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




31
Epoch 1/1
254631/254631 [==============================] - 406s 2ms/step - loss: 2.8330 - acc: 0.2734

Epoch 00001: acc improved from 0.26817 to 0.27335, saving model to /content/gdrive/My Drive/acc:0.273.hdf5



32
Epoch 1/1
254631/254631 [==============================] - 392s 2ms/step - loss: 2.8172 - acc: 0.2761

Epoch 00001: acc improved from 0.27335 to 0.27607, saving model to /content/gdrive/My Drive/acc:0.276.hdf5



33
Epoch 1/1
254631/254631 [==============================] - 388s 2ms/step - loss: 2.7989 - acc: 0.2815

Epoch 00001: acc improved from 0.27607 to 0.28146, saving model to /content/gdrive/My Drive/acc:0.281.hdf5



34
Epoch 1/1
254631/254631 [==============================] - 387s 2ms/step - loss: 2.7772 - acc: 0.2858

Epoch 00001: acc improved from 0.28146 to 0.28583, saving model to /content/gdrive/My Drive/acc:0.286.hdf5



35
Epoch 1/1
254631/254631 [==============================] - 386s 2ms/step - loss: 2.7646 - acc: 0.2883

Epoch 00001: acc improved f

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.351.hdf5')

In [0]:
Epoch = 61
while Epoch <= 100:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




61
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
254631/254631 [==============================] - 475s 2ms/step - loss: 2.4938 - acc: 0.3499

Epoch 00001: acc improved from -inf to 0.34995, saving model to /content/gdrive/My Drive/acc:0.350.hdf5



62
Epoch 1/1
254631/254631 [==============================] - 464s 2ms/step - loss: 2.4758 - acc: 0.3553

Epoch 00001: acc improved from 0.34995 to 0.35525, saving model to /content/gdrive/My Drive/acc:0.355.hdf5



63
Epoch 1/1
254631/254631 [==============================] - 456s 2ms/step - loss: 2.4695 - acc: 0.3568

Epoch 00001: acc improved from 0.35525 to 0.35679, saving model to /content/gdrive/My Drive/acc:0.357.hdf5



64
Epoch 1/1
254631/254631 [==============================] - 454s 2ms/step - loss: 2.4625 - acc: 0.3579

Epoch 00001: acc improved from 0.35679 to 0.35792, saving model to /content/gdrive/My Drive/acc:0.358.hdf5



65
Epoch 1/1
254631/254631 [============

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.401.hdf5')

In [0]:
Epoch = 101
while True:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




101
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
254631/254631 [==============================] - 398s 2ms/step - loss: 2.2949 - acc: 0.4006

Epoch 00001: acc improved from -inf to 0.40060, saving model to /content/gdrive/My Drive/acc:0.401.hdf5



102
Epoch 1/1
254631/254631 [==============================] - 390s 2ms/step - loss: 2.2841 - acc: 0.4021

Epoch 00001: acc improved from 0.40060 to 0.40208, saving model to /content/gdrive/My Drive/acc:0.402.hdf5



103
Epoch 1/1
254631/254631 [==============================] - 390s 2ms/step - loss: 2.2863 - acc: 0.4021

Epoch 00001: acc improved from 0.40208 to 0.40214, saving model to /content/gdrive/My Drive/acc:0.402.hdf5



104
Epoch 1/1
254631/254631 [==============================] - 390s 2ms/step - loss: 2.2788 - acc: 0.4032

Epoch 00001: acc improved from 0.40214 to 0.40324, saving model to /content/gdrive/My Drive/acc:0.403.hdf5



105
Epoch 1/1
254631/254631 [=======

KeyboardInterrupt: ignored

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.408.hdf5')

In [0]:
Epoch = 108
while True:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




108
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
254631/254631 [==============================] - 523s 2ms/step - loss: 2.2740 - acc: 0.4066

Epoch 00001: acc improved from -inf to 0.40659, saving model to /content/gdrive/My Drive/acc:0.407.hdf5



109
Epoch 1/1
254631/254631 [==============================] - 511s 2ms/step - loss: 2.2608 - acc: 0.4086

Epoch 00001: acc improved from 0.40659 to 0.40859, saving model to /content/gdrive/My Drive/acc:0.409.hdf5



110
Epoch 1/1
254631/254631 [==============================] - 499s 2ms/step - loss: 2.2568 - acc: 0.4109

Epoch 00001: acc improved from 0.40859 to 0.41090, saving model to /content/gdrive/My Drive/acc:0.411.hdf5
Seed:
" F#4B-3F#3F#5F#4F#56.116.11E-3F#56.10.14.10C#3F#511.3.611.3B2G5G41.4.8E2G#5G#4F#2B-3E3B1B311.3B311.3B311.3E2B38.11B38.11B38.11E-2B36.11B36.11B36.11G#2B311.4B311.4B311.4F#2B311.3B311.3B311.3E2B310.11.1B310.11.1B310.11.1E-2B3B2B3B2B3B2B1B39.11.3B39.11

In [0]:
model.load_weights('/content/gdrive/My Drive/acc:0.411.hdf5')

In [0]:
Epoch = 112
while True:
    print('\n\n')
    print(Epoch)
    model.fit(X, y, epochs=1,verbose=1, batch_size=128, callbacks=callbacks_list)
    if Epoch % 10 == 0:
      generate_notes(model, Epoch)
    Epoch+=1




112
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
254631/254631 [==============================] - 407s 2ms/step - loss: 2.2651 - acc: 0.4093

Epoch 00001: acc improved from -inf to 0.40933, saving model to /content/gdrive/My Drive/acc:0.409.hdf5



113
Epoch 1/1
254631/254631 [==============================] - 402s 2ms/step - loss: 2.2464 - acc: 0.4118

Epoch 00001: acc improved from 0.40933 to 0.41180, saving model to /content/gdrive/My Drive/acc:0.412.hdf5



114
Epoch 1/1
254631/254631 [==============================] - 399s 2ms/step - loss: 2.2468 - acc: 0.4124

Epoch 00001: acc improved from 0.41180 to 0.41240, saving model to /content/gdrive/My Drive/acc:0.412.hdf5



115
Epoch 1/1
254631/254631 [==============================] - 395s 2ms/step - loss: 2.2431 - acc: 0.4135

Epoch 00001: acc improved from 0.41240 to 0.41345, saving model to /content/gdrive/My Drive/acc:0.413.hdf5



116
Epoch 1/1
125056/254631 [=======

### I feel like it's a pretty good spot to stop here. 